<a href="https://colab.research.google.com/github/jinsusong/NLP-BERT-Review/blob/main/BERT_%ED%95%99%EC%8A%B5%EB%B0%A9%EB%B2%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Raw text에서 Training Instance 까지 

In [ ]:
# urllib모듈을 통해 가져온 문자열이 byte형태이기 때문에 데이터를 Decoding 해줘야 사람이 볼 수 있는 형태가 됨
import six

def convert_to_unicode(text):

    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")  # Deocde
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else:
        raise ValueError("Not running on Python 3?")

### Raw Data > Plain Text

- Raw Data란 텍스트만이 아닌 다른 정보들이나 html, 문자, 기호 등이 섞여 있는 데이터를 의미

- 파싱 (Parsing) : 모델 학습을 위한 데이터 전처리의 첫 번째 단계는 날것의 데이터에서 학습시킬 데이터만을 추출하는 것

- 파싱의 과정 
    1. 데이터 정제 ( html 제거 ) 
    2. 필요 없는 부분 삭제 ( meta 데이터 변형 ) 
    3. 학습하는 과정에서 필요한 정보 추가 (모델이 알 수 있도록 문서 , 문장을 구분하는 기호 추가 )

### Plain Text > Shard

- Sharding 이란 여러 Plain Text 를 여러 파일들로 나누는 작업을 의미

1. 멀티 프로세스로 처리하기 용이
2. 데이터를 섞는 효과

 - 데이터를 섞는다는 것은, 여러 출처의 자료들은 섞어서 하나의 Shard (작은 Plain Text) 를 만드는 것을 의미
 - 여러 출처의 자료를 섞어야 하는 이유는 언어 모델을 사전 학습시킬 때는 General 함이 유지되어야 하기 때문
 - 만약 데이터를 출처별로 순차적으로 학습한다면 그 데이터에 모델이 과적합 될 수 있다.

 

### Shard( Plain Text ) > Tokens

 - Basic Tokenizer
    1. 텍스트를 토큰화 하기 전에 Cleaning하는 과정을 거침 
        1. 문단 단위 (위의 예시에서 [DOC_SEP]) 로 텍스트들을 나눈다.
        2. 유효하지 않은 Character 들을 제거하고 Whitespace를 모두 띄어쓰기로 바꾼다.

In [ ]:
import unicodedata
    
    
def clean_text(text):
    """Performs invalid character removal and whitespace cleanup on text."""
    output = []
    for char in text:
        cp = ord(char)
        if cp == 0 or cp == 0xfffd or is_control(char):
            continue
        if is_whitespace(char):
            output.append(" ")
        else:
            output.append(char)
    return "".join(output)
        
 
 def is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    # \t, \n, and \r are technically control characters but we treat them
    # as whitespace since they are generally considered as such.
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False


def _is_control(char):
    """Checks whether `chars` is a control character."""
    # These are technically control characters but we count them as whitespace
    # characters.
    if char == "\t" or char == "\n" or char == "\r":
        return False
    cat = unicodedata.category(char)
    if cat.startswith("C"):
        return True
    return False

    2. 특수한 언어들을 처리한다.
        1. 예시로 중국어의 경우 한자 하나가 한 단어를 뜻하지만 띄어쓰기가 되어있지 않다. 따라서 한자의 양 옆에 빈칸을 넣어준다.

In [ ]:
def tokenize_chinese_chars(text):
    """Adds whitespace around any CJK character."""
    
    output = []
    for char in text:
        cp = ord(char)
        if _is_chinese_char(cp):
            output.append(" ")
            output.append(char)
            output.append(" ")
        else:
            output.append(char)
            
    return "".join(output)
    
    
def is_chinese_char(cp):
    """Checks whether CP is the codepoint of a CJK character."""
    # This defines a "chinese character" as anything in the CJK Unicode block:
    #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
    #
    # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
    # despite its name. The modern Korean Hangul alphabet is a different block,
    # as is Japanese Hiragana and Katakana. Those alphabets are used to write
    # space-separated words, so they are not treated specially and handled
    # like the all of the other languages.
    
    if ((0x4E00 <= cp <= 0x9FFF) or
            (0x3400 <= cp <= 0x4DBF) or
            (0x20000 <= cp <= 0x2A6DF) or
            (0x2A700 <= cp <= 0x2B73F) or
            (0x2B740 <= cp <= 0x2B81F) or
            (0x2B820 <= cp <= 0x2CEAF) or
            (0xF900 <= cp <= 0xFAFF) or
            (0x2F800 <= cp <= 0x2FA1F)):
        return True
        
    return False

    3.문장 부호를 고려하여 문장을 단어로 분리한다.

In [ ]:
import unicodedata


def _run_split_on_punc(token):
    """Splits punctuation on a piece of text."""
    if token in self.never_split:  # Don't split
        return [token]
    chars = list(token)
    i = 0
    start_new_word = True
    output = []
    while i < len(chars):
        char = chars[i]
        if is_punctuation(char):
            output.append([char])
            start_new_word = True
        else:
            if start_new_word:
                output.append([])
            start_new_word = False
            output[-1].append(char)
        i += 1
    return ["".join(x) for x in output]
       
       
def is_punctuation(char):
    """Checks whether `chars` is a punctuation character."""
    cp = ord(char)
    # We treat all non-letter/number ASCII as punctuation.
    # Characters such as "^", "$", and "`" are not in the Unicode
    # Punctuation class but we treat them as punctuation anyways, for
    # consistency.
    if (33 <= cp <= 47) or (58 <= cp <= 64) or (91 <= cp <= 96) or (123 <= cp <= 126):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

In [ ]:
- Subword Tokenizer (Wordpiece)
 1. Basic Tokenizer 을 거친 후 Subword Token으로 바꾸는 과정을 거치는데, 이를 Subword Tokenize라고 함 
 2. 언어 모델은 Vocab이라고 하는 Token의 리스트를 가짐
 3. 언어 모델은 자신의 Vocab 외에 있는 토큰들은 모두 [UNK] 토큰으로 처리
 4. 모델을 학습시킬 데이터들로 모델의 Vocab을 만들어야 함.
 5. BERT를 위한 Vocab는 Hugging Face의 BERT Tokenizer을 사용하여 만들 수 있음
 6. 

In [ ]:
import argparse
import json
from tokenizers import BertWordPieceTokenizer
from glob import glob

parser = argparse.ArgumentParser()

parser.add_argument('--vocab_size', type=int, default=32000)
parser.add_argument('--limit_alphabet', type=int, default=6000)

args = parser.parse_args()

tokenizer = BertWordPieceTokenizer(
    vocab=None,
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False,
    lowercase=False,
    wordpieces_prefix="##"
)

data = glob('/data/nlp/Merged/**training**.txt')  # 텍스트 데이터들

tokenizer.train(
    files=data,
    limit_alphabet=args.limit_alphabet,
    vocab_size=args.vocab_size
)

tokenizer.save("./ch-{}-wpm-{}-pretty".format(args.limit_alphabet, args.vocab_size), True)

vocab_path = "ch-6000-wpm-32000-pretty"
vocab_file = 'vocab.txt'

f = open(vocab_file, 'w', encoding='utf-8')
with open(vocab_path) as json_file:

    json_data = json.load(json_file)
    for item in json_data["model"]["vocab"].keys():
        f.write(item+'\n')

    f.close()

 - 언어 모델이 Vocab을 가지고 텍스트를 Tokenize 하는 이유
    1. 텍스트가 모델 학습에 사용되기 위해선 Encoding 과 Embedding의 과정을 거쳐야 함
    2. Encoding은 각 토큰을 독립적인 값으로 바꾸는 것을 의미
    3. 언어 모델이 Vocab을 가지고 텍스트를 Tokenize 하는 이유는 인간의 모든 단어를 Encoding 하는 것이 불가능하기 때문
    4. 세상에 존재하는 모든 단어들의 숫자는 무한대로 발산하기 때문에 모든 각 단어들에 다른 Index를 부여할 수 없다. 따라서 모델이 처리할 수 있는 (아는) 단어를 정해주는데, 이를 Vocab이라고 하는 것이다.
    
 -  그러면 모델이 처리할 수 있는 단어들만 따로 저장해두면 되지, 왜 단어들을 분리하는 것일까? 
    1. Tokenize는 크게 Word Tokenizer 과 Subword Tokenizer로 구분
    2. Word Tokenizer는 한 단어를 쪼개지 않고 토큰화 하는 것을 의미하며 Subword Tokenizer는 한 단어를 여러 개의 토큰으로 바꾸는 것을 의미

 - 최근에는 대부분 Subword Tokenizer 가 사용되는데 그 이유는
    1. 경우의 수를 줄여준다 (토큰의 재사용이 가능하다)
    2. OOV (Out of Vocabulary) 처리에 용이

- BERT에서는 Wordpiece로 Vocab을 만든 뒤 Greedy Longest-Match-First Algorithm 방식을 사용하여 텍스트들을 Token 으로 바꿈

- Greedy Longest-Match-First Algorithm
 1. [설명]

 2. 각 단어를 토큰화 할 때 Start (S) 와 End (E) 가 있다.
 3. S, E는 각 단어의 Character Index를 의미한다.

 4. 단어 (S, E) : 타겟 -> 결과
 5. 타겟은 단어[S:E] 이다.
 6. 결과는 해당 토큰이 Vocab에 있는지 없는지이다. (있으면 O / 없으면 X)

 7. 결과가 X 이면 E를 1을 줄인다.
 8. 결과가 O 이면 당시 E 값을 S에 주고 다시 끝 Index부터 시작한다.
 9. 결과가 O 일 때 S가 0이 아니면 앞에 ##을 붙인다.
 10. 예시로 unaffable 이라는 단어를 토큰화 해보자.
 
 11. [예시]

 12. unaffable (0, 9) : unaffable → X

 13. unaffable (0, 8) : unaffabl → X

 14. ...


 15. un (0, 2) : un → O  => un

 16. unaffable (2, 9) : affable → X

 17. unaffable (2, 8) : affabl → X

 18. ...


 19. aff (2, 5) : aff → O => ##aff

 20. able (5, 9) : able → O => ##able

### Tokens > [CLS] Seq_A [SEP] Seq_B [SEP]

 - BERT의 사전학습은 MASKED 된 토큰을 맞추고 다음 A와 B가 이어지는 Sequence인지 맞추는 과정
 - 토큰들을 모델이 학습할 수 있는 정보들을 담은 하나의 객체로 저장해야함
 - 저장하는 확장자로는 npy, hdf5, lmdb  크기가 큰 데이터들을 저장하는데 용이한 확장자
 

- Target Sequence Length
    1. BERT의 Input 형태는 [CLS] Seq_A [SEP] Seq_B [SEP]
    2. Seq_A 와 Seq_B 는 일련의 토큰들의 집합이다. 따라서 A 와 B에 각각 몇 개의 토큰이 들어갈지 결정해주어야 함
    3. Input의 Max Sequence Length가 128이라면, [CLS] 1개와 [SEP] 2개를 제외하면 총 125개의 토큰을 채워야 함.
    4. 다만 BERT에서는 학습의 다양성을 위해 0.1의 확률로 Target Sequence Length를 2 이상 127 이하의 정수로 선택하게 된다.
    

In [ ]:
import random

rng = random.Random(args.random_seed)  # Give random a seed (To maintain the result)

target_seq_length = max_num_tokens - 3  # 125 = 128 - 3

if rng.random() < 0.1: 
    target_seq_length = rng.randint(2, max_num_tokens)  # 0.1의 확률로 125보다 작은 것

 - 토큰들의 개수가 Target Sequence Length를 넘을 때까지 문장들을 더함

한 문서가 총 26개의 문장 (S1, S2, .. S26)으로 이루어졌으며, 각 문장은 10개의 토큰들로 이루어진 상황을 가정해보자.

그러면 S1부터 각 토큰의 합이 Target Sequence Length 를 넘을 때까지 Chunk에 이어서 넣는다. Chunk는 그냥 Input의 후보라고 생각하면 된다. 그러면 현재 Chunk에는 S13가지 들어가 있을 것이다. 

In [ ]:
current_chunk = []
current_length = 0
i = 0

while i < len(document):  # 한 문서에 있는 문장의 개수
    segment = document[i]  # 한 문장
    current_chunk.append(segment)  # 한 문장 append
    current_length += len(segment)  # 한 문장에 있는 토큰의 개수를 더함

    # 마지막 문장이거나 거나 / 현재 청크의 토큰 개수가 이미 target 이상이면 (보통 125)
    if i == len(document) - 1 or current_length >= target_seq_length:
    	break
    else:
    	i += 1

- Chunk의 문장들 중 분기점을 정함


Chunk 에 있는 문장들 중 랜덤으로 분기점을 정한다. 위의 예시에서는 1~13 사이의 숫자가 분기점이 될 것이다. 만약 분기점이 6이었다면, S1 ~ S5 가 Seq_A 로 들어가게 된다.

In [ ]:
# `a_end` is how many segments from `current_chunk` go into the `A`
if len(current_chunk) >= 2:  # number of sentences added to chunk
	a_end = rng.randint(1, len(current_chunk) - 1)  # 분기점을 정함
else:
	a_end = 1

seq_a = []
for j in range(a_end):  # 분리점으로 이전 segment 들은 Token_A 에 포함
	seq_a.extend(current_chunk[j])

- Random Next 유무
 1.  Random Next가 아닌 경우
    1. Seq_A에 들어가지 않은 Chunk의 나머지 문장들을 Seq_B에 넣는다. 위의 예시에서는 S6 ~ S13이  Seq_B에 들어갈 것이다.
 2. Random Next인 경우
    1. BERT 에서는 0.5의 확률로 Seq_A 다음에 랜덤 토큰들이 Seq_B로 오게 된다.이때 Seq_B를 정하는 방법은 완전 다른 문서에서 일련의 토큰들을 가져오는 것이다.

In [ ]:
# Random Next가 아닌 경우
is_random_next = False
for j in range(a_end, len(current_chunk)):
	seq_b.extend(current_chunk[j])
 
 # Random Next인 경우

 if len(current_chunk) == 1 or rng.random() < 0.5:  # 청크에 문장이 1개이거나 50%의 확률로 다른 문장 이어붙임
    
    is_random_next = True
    target_b_length = target_seq_length - len(seq_a)  # Seq_B 에 들어갈 토큰의 개수를 구한다

    for _ in range(10):  # B에 들어갈 토큰들을 가져올 문서를 정함
        random_document_index = rng.randint(0, len(all_documents) - 1)  # 무작위 문서
        if random_document_index != document_index:  # A를 가져온 문서와 다른 문서여야 한다
            break 

    # If picked random document is the same as the current document
    if random_document_index == document_index:
        is_random_next = False  # 혹여나 같으면 random_next를 False로 만든다

    random_document = all_documents[random_document_index]
    random_start = rng.randint(0, len(random_document) - 1)  # 고른 문서에서 랜덤 시작점 고름

    for j in range(random_start, len(random_document)):
        tokens_b.extend(random_document[j])  # 토큰화된 문장 하나씩 넣는다
        if len(seq_b) >= seq_b_length:  # Seq_B
            break

    # 분기점 이후로 문장들은 실제로 안썼다 (S6~S13) -> 다시 보관한다
    num_unused_segments = len(current_chunk) - a_end
    i -= num_unused_segments

 - Truncate
    1. Seq_A + Seq_B 가 Target Sequence Length 보다 길면 Truncate 하는 과정을 거친다.
    2. Truncate 할 때는 A와 B 중 더 긴 것의 앞이나 뒤에서 토큰을 하나씩 제거해준다.

In [ ]:
def truncate_seq_pair(seq_a, seq_b, target_seq_length, rng):
    """Truncates a pair of sequences to a maximum sequence length."""
    
    while True:
        total_length = len(seq_a) + len(seq_b)  
        if total_length <= target_seq_length:  # 합이 target_seq_length 보다 클 때
            break

        trunc_tokens = seq_a if len(seq_a) > len(seq_b) else seq_b  # 더 긴 것 선택
        assert len(trunc_tokens) >= 1

        # We want to sometimes truncate from the front and sometimes from the
        # back to add more randomness and avoid biases.
        if rng.random() < 0.5:  # 0.5의 확률로 
            del trunc_tokens[0]  # 맨 앞 토큰 제거 
        else:
        	trunc_tokens.pop()  # 맨 뒤 토큰 제거
 

### [CLS] Seq_A [SEP] Seq_B [SEP] > Masked Sequence

- 이제 남은 것은 일부 토큰들을 Masking 하는 것

In [ ]:
def create_masked_lm_predictions(tokens, masked_lm_prob,  max_predictions_per_seq, vocab_words, rng):
    """Creates the predictions for the masked LM objective."""

    cand_indexes = []
    for (i, token) in enumerate(tokens):
        if token == "[CLS]" or token == "[SEP]":  # [CLS], [SEP]는 마스킹 후보에서 제외
            continue
        cand_indexes.append(i)

    rng.shuffle(cand_indexes)

    output_tokens = list(tokens)

    # 몇개를 마스킹 할 것이냐 -> 토큰 개수 * 0.15 (최대 20개)
    num_to_predict = min(max_predictions_per_seq, max(1, int(round(len(tokens) * masked_lm_prob))))

    masked_lms = []  # 마스킹 된 토큰 객체 (인데스와 라벨이 포함되어있다)
    covered_indexes = set()  # 마스킹할 토큰의 인덱스
    for index in cand_indexes:
        if len(masked_lms) >= num_to_predict:  # 목표치 채웠으면 끝내고
            break
        if index in covered_indexes:  # 이미 마스킹된 index 면 건너뛰기
            continue
        covered_indexes.add(index)  # 해당 Index 마스킹 할 것이다.

        # 80% of the time, replace with [MASK]
        if rng.random() < 0.8:
            masked_token = "[MASK]"  # 0.8의 확률로 마스킹
        else:
            if rng.random() < 0.5:
                masked_token = tokens[index]  # 0.1의 확률로 그대로
            else:
                masked_token = vocab_words[rng.randint(0, len(vocab_words) - 1)]  # 0.1의 확률로 랜덤 토큰

        output_tokens[index] = masked_token  # 마스킹된 토큰으로 교체
        masked_lms.append(MaskedLmInstance(index=index, label=tokens[index]))  # 마스킹한 토큰의 인덱스와 라벨 저장

    masked_lms = sorted(masked_lms, key=lambda x: x.index)  # Index 로 정렬

    masked_lm_positions = []
    masked_lm_labels = []

    for p in masked_lms:
        masked_lm_positions.append(p.index)  # 마스킹한 토큰의 인덱스 저장
        masked_lm_labels.append(p.label)  # 마스킹한 토큰의 라 저장

    # (전체 토큰들 - 마스킹 반영, 마스킹 된 토큰의 위치, 마스킹된 토큰의 라)
    return output_tokens, masked_lm_positions, masked_lm_labels

 - Seq_A와  Seq_B의 토큰들 중 몇 개를 마스킹할지 우선 정한다.
 - 랜덤으로 해당 개수만큼 토큰을 골라 마스킹하고 정답과 그 토큰의 위치를 저장한다. 
 - 특이점은 마스킹할 토큰 중 0.8의 확률로 [MASK], 0.1의 확률로 그대로, 0.1의 확률로 다른 토큰 (랜덤)으로 바꾼다는 것이다.

### Masked Sequence > Encoding, Padding

- 모든 토큰들 ( [CLS] Seq_A [SEP] Seq_B [CLS] / 마스킹된 토큰의 라벨들 )을 Vocab에서의 Index로 바꾼다.
- Masked SequencE

- [CLS] [MASK] ##가 방 ##에 [UNK] 들 ##어 [Mask] ##셨 ##다 [SEP] 코딩 ##은 재미 ##있 ##다 [SEP]


- Encoded Sequence

-  2      4    523  8 312   1   53 5234   4   323  123   3    21  33  22   21  123   3
 
 
- Original Label
 
- 1, 8
 
 
- Encoded Label

- 3444, 553

또한 전체 Sequence의 길이가 128 보다 작으면 남은 공간들을 [PAD] 토큰으로 채운다.

In [ ]:
while len(masked_lm_positions) < max_predictions_per_seq:  # Fill empty space in tokens [PAD]
	
    masked_lm_positions.append(0)
	masked_lm_ids.append(0)

###Encoded & Padded Sequence > Training Instance

- Training Instance에 들어가는 정보
    1. 마스킹 된 토큰들의 id
    2. 마스킹 된 토큰의 위치
    3. 마스킹 된 코튼의 라벨
    4. Random Next (True / False)
    5. 0 ... 0 , 1 ... 1 (Seq_A 와 Seq_B 구분)